In [ ]:
#from pyunpack import Archive
#Archive('/content/drive/MyDrive/Ata.rar').extractall('/content/drive/MyDrive/')

In [ ]:
import numpy as np
import pandas as pd
import cv2
from tensorflow import keras
from tqdm import tqdm

In [ ]:
input = keras.layers.Input(shape=(42, 175, 1))
x = keras.layers.Conv2D(32, (5,5),padding='valid',activation='relu')(input)
x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = keras.layers.Conv2D(64, (3,3),padding='same',activation='relu')(x)
x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = keras.layers.Conv2D(128, (3,3),padding='same',activation='relu')(x)
maxpool = keras.layers.MaxPooling2D((2,2),padding='same')(x)
outputs = []
for i in range(5):
    x = keras.layers.Conv2D(256, (3,3),padding='same',activation='relu')(maxpool)
    x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(10 , activation='softmax' , name=f'char_{i+1}')(x)
    outputs.append(x)
    
model = keras.models.Model(inputs = input , outputs=outputs)

In [ ]:
import os

In [ ]:
filedir = '/content/drive/MyDrive/Ata/'

In [ ]:
lidir = [filedir+i for i in os.listdir(filedir)]

In [ ]:
y = np.zeros((len(lidir), 5, 10))

In [ ]:
X = []

In [ ]:
for i in tqdm(range(len(lidir))):
    img = cv2.imread(lidir[i], 0)
    img = np.array(img)/255.0
    X.append(img)
    a = lidir[i].split('/')[-1].split('.')[0]
    for j in range(len(a)):
        y[i][j][int(a[j])] = 1
X = np.array(X).reshape(-1, 42, 175, 1)

100%|██████████| 991/991 [00:02<00:00, 474.04it/s]


In [ ]:
y1 = y[:, 0, :]
y2 = y[:, 1, :]
y3 = y[:, 2, :]
y4 = y[:, 3, :]
y5 = y[:, 4, :]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001), loss_weights=[1,1,1,1,1], metrics=['accuracy'])

In [ ]:

hist = model.fit(x=X, y=[y1, y2, y3, y4, y5], epochs=100, validation_split=0.2)

Epoch 1/100
25/25 [==============================] - 5s 54ms/step - loss: 0.1050 - char_1_loss: 0.0115 - char_2_loss: 0.0179 - char_3_loss: 0.0220 - char_4_loss: 0.0346 - char_5_loss: 0.0190 - char_1_accuracy: 0.9974 - char_2_accuracy: 0.9976 - char_3_accuracy: 0.9982 - char_4_accuracy: 0.9972 - char_5_accuracy: 0.9973 - val_loss: 1.4407 - val_char_1_loss: 0.2141 - val_char_2_loss: 0.4345 - val_char_3_loss: 0.3694 - val_char_4_loss: 0.2788 - val_char_5_loss: 0.1440 - val_char_1_accuracy: 0.9397 - val_char_2_accuracy: 0.8744 - val_char_3_accuracy: 0.9196 - val_char_4_accuracy: 0.9296 - val_char_5_accuracy: 0.9397
Epoch 2/100
25/25 [==============================] - 1s 25ms/step - loss: 0.0851 - char_1_loss: 0.0178 - char_2_loss: 0.0189 - char_3_loss: 0.0129 - char_4_loss: 0.0175 - char_5_loss: 0.0180 - char_1_accuracy: 0.9986 - char_2_accuracy: 0.9991 - char_3_accuracy: 0.9999 - char_4_accuracy: 0.9985 - char_5_accuracy: 0.9986 - val_loss: 1.4625 - val_char_1_loss: 0.2020 - val_char_2_l

In [ ]:
model.save('ata.hdf5')

In [ ]:
model.predict(X[0].reshape(-1, 42, 175, 1))

[array([[8.4952359e-07, 8.6542883e-08, 9.9999321e-01, 1.0793406e-07,
         2.6471932e-06, 3.0591454e-07, 3.2190908e-07, 1.4953762e-06,
         7.4779967e-07, 2.2761080e-07]], dtype=float32),
 array([[3.5061797e-07, 2.0248791e-07, 2.5604857e-07, 2.3710372e-06,
         3.0507643e-08, 9.9999392e-01, 7.6401744e-07, 8.4302968e-07,
         1.7525657e-07, 1.1805662e-06]], dtype=float32),
 array([[9.9998844e-01, 1.0779884e-06, 1.0338633e-07, 9.4759093e-07,
         3.4317145e-07, 3.1954320e-07, 3.3360450e-06, 1.9587933e-06,
         8.6099249e-08, 3.3477481e-06]], dtype=float32),
 array([[9.8827407e-08, 2.1367402e-06, 2.8580462e-07, 2.0294918e-07,
         9.9999571e-01, 4.9991481e-08, 1.1166619e-07, 4.2295252e-07,
         1.1969321e-07, 9.6622773e-07]], dtype=float32),
 array([[2.5072057e-07, 1.8775795e-07, 9.9999070e-01, 4.7763848e-07,
         5.5897436e-07, 2.2524704e-07, 9.3694055e-08, 2.2955178e-06,
         8.1848896e-07, 4.2921638e-06]], dtype=float32)]

In [ ]:
np.argmax(model.predict(X[0].reshape(-1, 42, 175, 1)), axis=0)

array([[2, 3, 0, 1, 3, 1, 2, 4, 4, 4]])

In [ ]:
pred = model.predict(X[0].reshape(-1, 42, 175, 1))

In [ ]:
li = []
for i in pred:
  li.append(str(np.argmax(i)))
string = ''.join(li)

In [ ]:
print(string)

25042


In [ ]:
li

['2']

In [ ]:
import cv2
from tensorflow import keras
input = keras.layers.Input(shape=(42, 175, 1))
x = keras.layers.Conv2D(32, (5,5),padding='valid',activation='relu')(input)
x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = keras.layers.Conv2D(64, (3,3),padding='same',activation='relu')(x)
x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
x = keras.layers.Conv2D(128, (3,3),padding='same',activation='relu')(x)
maxpool = keras.layers.MaxPooling2D((2,2),padding='same')(x)
outputs = []
for i in range(5):
    x = keras.layers.Conv2D(256, (3,3),padding='same',activation='relu')(maxpool)
    x = keras.layers.MaxPooling2D((2,2),padding='same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(10 , activation='softmax' , name=f'char_{i+1}')(x)
    outputs.append(x)
model = keras.models.Model(inputs = input , outputs=outputs)
model.load_weights('/content/ata.hdf5')
def read_captcha(img_add):#gets the img full address
  img = cv2.imread(img_add, 0)
  img = np.array(img)/255.0
  pred = model.predict(img.reshape(-1, 42, 175, 1))
  li = []
  for i in pred:
    li.append(str(np.argmax(i)))
  string = ''.join(li)
  return string

In [ ]:
read_captcha('/content/drive/MyDrive/Ata/00642.jpg')

'00642'